# Extracting frames and landmarks from dataset

In [1]:
%load_ext cudf.pandas

In [2]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
from pymongo import MongoClient
import gc

In [23]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(static_image_mode = False,
                                model_complexity = 2)

I0000 00:00:1730028763.545234  402030 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1730028763.618257  410082 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.107.02), renderer: NVIDIA GeForce RTX 4050 Laptop GPU/PCIe/SSE2


W0000 00:00:1730028763.682152  410065 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730028763.771491  410076 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730028763.773721  410066 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730028763.773779  410079 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730028763.773781  410076 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730028763.780717  410074 inference_feedback_manager.cc:114] Feedback manager 

In [4]:
connection = MongoClient('localhost', 27017)
db = connection['mydb']
collection = db['Sign_Language_Final_Data']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
cursor = collection.find({})
df = pd.DataFrame(list(cursor))

W0000 00:00:1730027090.221653  402124 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730027090.317357  402137 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730027090.319724  402125 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730027090.319900  402138 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730027090.322865  402122 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730027090.326819  402138 inference_feedback_manager.cc:114] Feedback manager 

In [6]:
df.head()

,_id,gloss,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,is_available
0,671c624d104a133c1d8d0b1c,book,"[385, 37, 885, 720]",25,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0,v_id_69241,True
1,671c624d104a133c1d8d0b1d,book,"[462, 44, 949, 720]",25,-1,1,10,31,signschool,train,https://signstock.blob.core.windows.net/signsc...,0,v_id_07069,True
2,671c624d104a133c1d8d0b1e,book,"[234, 17, 524, 414]",25,-1,1,17,36,startasl,train,https://s3-us-west-1.amazonaws.com/files.start...,0,v_id_07068,True
3,671c624d104a133c1d8d0b1f,book,"[131, 26, 526, 480]",25,-1,1,22,59,asldeafined,train,https://media.asldeafined.com/vocabulary/14666...,0,v_id_07070,True
4,671c624d104a133c1d8d0b20,book,"[162, 54, 528, 400]",25,-1,1,24,12,aslsearch,val,http://www.aslsearch.com/signs/videos/book.mp4,0,v_id_07099,True


In [7]:
df.drop(columns = ['_id', 'source', 'url'], inplace = True)

In [8]:
df.head()

,gloss,bbox,fps,frame_end,frame_start,instance_id,signer_id,split,variation_id,video_id,is_available
0,book,"[385, 37, 885, 720]",25,-1,1,0,118,train,0,v_id_69241,True
1,book,"[462, 44, 949, 720]",25,-1,1,10,31,train,0,v_id_07069,True
2,book,"[234, 17, 524, 414]",25,-1,1,17,36,train,0,v_id_07068,True
3,book,"[131, 26, 526, 480]",25,-1,1,22,59,train,0,v_id_07070,True
4,book,"[162, 54, 528, 400]",25,-1,1,24,12,val,0,v_id_07099,True


In [9]:
df['video_id'] = df['video_id'].apply(lambda id: id.replace('v_id_', ''))

In [10]:
df.head()

,gloss,bbox,fps,frame_end,frame_start,instance_id,signer_id,split,variation_id,video_id,is_available
0,book,"[385, 37, 885, 720]",25,-1,1,0,118,train,0,69241,True
1,book,"[462, 44, 949, 720]",25,-1,1,10,31,train,0,07069,True
2,book,"[234, 17, 524, 414]",25,-1,1,17,36,train,0,07068,True
3,book,"[131, 26, 526, 480]",25,-1,1,22,59,train,0,07070,True
4,book,"[162, 54, 528, 400]",25,-1,1,24,12,val,0,07099,True


In [11]:
len(df)

11980

In [12]:
df['frame_start'].unique().tolist()

[1, 2, 3]

In [13]:
df['frame_end'].unique().tolist()

[-1]

In [14]:
df['split'].unique().tolist()

['train', 'val', 'test']

In [15]:
(df['split'] == 'train').sum()

8313

In [16]:
(df['split'] == 'test').sum()

1414

In [17]:
(df['split'] == 'val').sum()

2253

In [18]:
df['gloss'].nunique()

1999

In [19]:
def count_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

In [20]:
def extract_landmarks(video_path, frame_start, frame_end):
    cap = cv2.VideoCapture(video_path)
    
    total_frames = count_frames(video_path)

    if frame_end == -1:
        frame_end = total_frames - 1

    landmarks_sequence = []
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_start)

    while cap.isOpened() and frame_start <= frame_end:
        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

        if current_frame > frame_end:
            break
        
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = holistic.process(image_rgb)

        frame_landmarks = {}

        if results.face_landmarks:
            frame_landmarks["face"] = [(lm.x, lm.y, lm.z) for lm in results.face_landmarks.landmark]
        if results.left_hand_landmarks:
            frame_landmarks["left_hand"] = [(lm.x, lm.y, lm.z) for lm in results.left_hand_landmarks.landmark]
        if results.right_hand_landmarks:
            frame_landmarks["right_hand"] = [(lm.x, lm.y, lm.z) for lm in results.right_hand_landmarks.landmark]
        
        landmarks_sequence.append(frame_landmarks)

    cap.release()
    return landmarks_sequence

In [21]:
import warnings
warnings.filterwarnings("ignore")

landmarks_data = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing videos"):
    video_id = row['video_id']
    frame_start = row['frame_start']
    frame_end = row['frame_end']
    video_path = f'./kaggle-dataset/videos/{video_id}.mp4'

    landmarks_sequence = extract_landmarks(video_path, frame_start, frame_end)

Processing videos:   2%|▏         | 224/11980 [24:37<21:32:37,  6.60s/it]


KeyboardInterrupt: 

In [22]:
# cap = cv2.VideoCapture(0)

# with mp_holistic.Holistic(static_image_mode=False, 
#                            model_complexity=2, 
#                            enable_segmentation=True,
#                            min_detection_confidence=0.5,
#                            min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():
#         ret,frame = cap.read()
#         if not ret:
#             print("Ignoring empty camera frame")
#             continue

#         frame = cv2.flip(frame, 1)
        
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False

#         results = holistic.process(image)

#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         if results.face_landmarks:
#              mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
#         # if results.pose_landmarks:
#         #      mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
#         if results.left_hand_landmarks:
#              mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
#         if results.right_hand_landmarks:
#              mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

#         cv2.imshow('TEST', image)

#         if cv2.waitKey(5) & 0xFF == 27:  # Press 'Esc' to exit
#                 break

# cap.release()
# cv2.destroyAllWindows()
